In [34]:
doc = """
This jupyter notebook is authored by ygg_anderson for the Token Engineering Commons. See appropriate licensing. 🐧 🐧 🐧
"""

import param
import panel as pn
import pandas as pd
import hvplot.pandas
import holoviews as hv
import numpy as np
from scipy.stats.mstats import gmean
pn.extension()


# Load CSTK data
cstk_data = pd.read_csv('CSTK_DATA.csv', header=None).reset_index().head(100)
cstk_data.columns = ['CSTK Token Holders', 'CSTK Tokens']
cstk_data['CSTK Tokens Capped'] = cstk_data['CSTK Tokens'].apply(lambda x: min(x, cstk_data['CSTK Tokens'].sum()/10))

In [35]:
cstk_data

,CSTK Token Holders,CSTK Tokens,CSTK Tokens Capped
0,0,176256,83367.7
1,1,112192,83367.7
2,2,46263,46263.0
3,3,44539,44539.0
4,4,30783,30783.0
...,...,...,...
95,95,570,570.0
96,96,565,565.0
97,97,563,563.0
98,98,560,560.0


# Hatch Template!

In [36]:
class Hatch(param.Parameterized):
    # CSTK Ratio
    total_cstk_tokens = param.Number(cstk_data['CSTK Tokens Capped'].sum(), constant=True)
    hatch_oracle_ratio = param.Number(0.005, bounds=(0.005, 100), step=0.005)
    
    # Min and Target Goals
    max_raise = param.Number(0.75, bounds=(0.5,1), step=0.01)
    min_raise = param.Number(0.05, bounds=(0.01,0.5), step=0.01)
    target_raise = param.Number(0.50, bounds=(0.01,1), step=0.01) 
    
    # Hatch params
    hatch_period_days = param.Integer(15, bounds=(5, 30), step=2)
    
    # Number of TESTTEC exchanged for 1 wxdai
    hatch_exchange_rate = param.Number(10000, bounds=(1,100000), step=1) 
    hatch_tribute = param.Number(0.05, bounds=(0,1))    
    
    def min_goal(self):
        return self.min_raise * self.total_cstk_tokens * self.hatch_oracle_ratio
    
    def max_goal(self):
        return self.max_raise * self.total_cstk_tokens * self.hatch_oracle_ratio

    def wxdai_range(self):
        return pn.Row(pn.Pane("Cap on wxdai staked: "), self.hatch_oracle_ratio * self.total_cstk_tokens)
    
    def hatch_raise_view(self):
        # Load CSTK data
        cstk_data = pd.read_csv('CSTK_DATA.csv', header=None).reset_index().head(100)
        cstk_data.columns = ['CSTK Token Holders', 'CSTK Tokens']
        cstk_data['CSTK Tokens Capped'] = cstk_data['CSTK Tokens'].apply(lambda x: min(x, cstk_data['CSTK Tokens'].sum()/10))
        cstk_data['Cap raise'] = cstk_data['CSTK Tokens Capped'] * self.hatch_oracle_ratio

        cap_plot = cstk_data.hvplot.area(title="Raise Targets Per Hatcher", x='CSTK Token Holders', y='Cap raise', yformatter='%.0f', label="Cap Raise", ylabel="XDAI Staked")

        cstk_data['max_goal'] = cstk_data['Cap raise'] * self.max_raise
        max_plot = cstk_data.hvplot.area(x='CSTK Token Holders', y='max_goal', yformatter='%.0f', label="Max Raise")

        cstk_data['min_goal'] = cstk_data['Cap raise'] * self.min_raise
        min_plot = cstk_data.hvplot.area(x='CSTK Token Holders', y='min_goal', yformatter='%.0f', label="Min Raise")

        cstk_data['target_goal'] = cstk_data['Cap raise'] * self.target_raise 
        target_plot = cstk_data.hvplot.line(x='CSTK Token Holders', y='target_goal', yformatter='%.0f', label="Target Raise")
        
        raise_bars = cstk_data.iloc[:,3:].sum().sort_values(ascending=False).hvplot.bar(yformatter='%.0f', title="Total Raise Targets")
        
        stats = pd.DataFrame(cstk_data.iloc[:,3:].sum(), columns=['Total XDAI Raise'])
        stats['GMean XDAI Co-vested Per Hatcher'] = gmean(cstk_data.iloc[:,3:])
        stats['XDAI Hatch Tribute'] = stats['Total XDAI Raise'] * self.hatch_tribute
        stats['Total TECH Tokens'] = stats['Total XDAI Raise'] * self.hatch_exchange_rate
        
        
        return pn.Column(cap_plot * max_plot * min_plot * target_plot, raise_bars, stats.sort_values('Total XDAI Raise',ascending=False).apply(round).reset_index().hvplot.table())

In [37]:
## Use range for min_goal, max_goal in the next notebook

class A(param.Parameterized):
    integer_range = param.Range(default=(3, 7), bounds=(0, 10))
pn.Pane(A)

Param(ParameterizedMetaclass, name='A')

In [38]:
h = Hatch()

pn.Row(h, h.hatch_raise_view)

Row
    [0] Column(margin=5, name='Hatch', width=300)
        [0] StaticText(value='<b>Hatch</b>')
        [1] FloatInput(disabled=True, name='Total cstk tokens', value=711964.4, value_throttled=711964.4)
        [2] FloatSlider(end=100, name='Hatch oracle ratio', start=0.005, step=0.005, value=0.005, value_throttled=0.005)
        [3] FloatSlider(name='Max raise', start=0.5, step=0.01, value=0.75, value_throttled=0.75)
        [4] FloatSlider(end=0.5, name='Min raise', start=0.01, step=0.01, value=0.05, value_throttled=0.05)
        [5] FloatSlider(name='Target raise', start=0.01, step=0.01, value=0.5, value_throttled=0.5)
        [6] IntSlider(end=30, name='Hatch period days', start=5, step=2, value=15, value_throttled=15)
        [7] FloatSlider(end=100000, name='Hatch exchange rate', start=1, step=1, value=10000, value_throttled=10000)
        [8] FloatSlider(name='Hatch tribute', value=0.05, value_throttled=0.05)
    [1] ParamMethod(method)

## Dandelion Voting

1. Percentage of total tokens that have to vote 'yes' to `something` for it to pass.

In [28]:
class DandelionVoting(param.Parameterized):
    total_tokens = param.Number(1e6, constant=True)
    support_required = param.Number(0.6, bounds=(0.5,0.9), step=0.01)
    minimum_accepted_quorum = param.Number(0.02, bounds=(0.01,1), step=0.01)
    vote_duration_days = param.Number(3, bounds=(1,14), step=1)
    vote_buffer_hours = param.Number(8, bounds=(1,48), step=1)
    rage_quit_hours = param.Number(24, bounds=(1, 48), step=1)
    tollgate_fee_xdai = param.Number(3, bounds=(1,100), step=1)
    
    
    def vote_pass_view(self):
        x = np.linspace(0, self.total_tokens, num=100)
        y = [a*self.support_required for a in x]
        df = pd.DataFrame(zip(x,y))
        y_fill = [a for a in x]
        df_fill = pd.DataFrame(zip(x,y_fill))
        y_fill_quorum = [a for i, a in enumerate(x) if i < self.minimum_accepted_quorum*len(x)]
        df_fill_q = pd.DataFrame(zip(x,y_fill_quorum))
        total_votes_plot = df_fill.hvplot.area(
                title = "Minimum Support and Quorum Accepted for Proposals to Pass", 
                x='0', y='1', xformatter='%.0f', yformatter='%.0f', color='green', 
                xlabel='Total Token Votes', ylabel='Token Parity (Vote Distribution)')
        support_required_plot = df.hvplot.area(x='0', y='1', xformatter='%.0f', yformatter='%.0f', color='red')
        quorum_accepted_plot = df_fill_q.hvplot.area(x='0', y='1', xformatter='%.0f', yformatter='%.0f', color='red')
        return total_votes_plot * support_required_plot * quorum_accepted_plot
            

### Dandelion Voting
The minimum quorum accepted is the percentage of **all tokens** that must vote yes. Support required is the percentage of **tokens that voted** that must vote yes. As the number of **No** votes grows, the **Yes** votes

In [29]:
d = DandelionVoting()
pn.Row(d, d.vote_pass_view)

Row
    [0] Column(margin=5, name='DandelionVoting', width=300)
        [0] StaticText(value='<b>DandelionVoting</b>')
        [1] FloatInput(disabled=True, name='Total tokens', value=1000000.0, value_throttled=1000000.0)
        [2] FloatSlider(end=0.9, name='Support required', start=0.5, step=0.01, value=0.6, value_throttled=0.6)
        [3] FloatSlider(name='Minimum accepted quorum', start=0.01, step=0.01, value=0.02, value_throttled=0.02)
        [4] FloatSlider(end=14, name='Vote duration days', start=1, step=1, value=3, value_throttled=3)
        [5] FloatSlider(end=48, name='Vote buffer hours', start=1, step=1, value=8, value_throttled=8)
        [6] FloatSlider(end=48, name='Rage quit hours', start=1, step=1, value=24, value_throttled=24)
        [7] FloatSlider(end=100, name='Tollgate fee xdai', start=1, step=1, value=3, value_throttled=3)
    [1] ParamMethod(method)

## Impact Hours

In [30]:
class ImpactHoursData(param.Parameterized):
    predicted = pd.read_csv('data/IHPredictions.csv').query('Model=="Historic"')
    optimistic =  pd.read_csv('data/IHPredictions.csv').query('Model=="Optimistic"')
    prediction = param.Number(0, bounds=(0,1), step=0.01)
    
    @param.depends('prediction')
    def view(self):
        x = 'End Date'
        y_curve = 'Total IH'
        y_bar = 'Total Hours'
        
        actual_curve = self.predicted.hvplot(x, y_curve, rot=45, title='Impact Hours Accumulation Curve :)')
        actual_bar = self.predicted.hvplot.bar(x, y_bar, label='Actual')
        
        predicted = self.predicted[self.predicted["Actual / Predicted"] == "Predicted"].copy()
        predicted[y_curve] = 0.5 * predicted[y_curve] + 0.5 * self.optimistic[self.optimistic["Actual / Predicted"] == "Predicted"][y_curve]
        
        predicted_curve = predicted.hvplot(x, y_curve, rot=45, title='Impact Hours Accumulation Curve :)')
        predicted_bar = predicted.hvplot.bar(x, y_bar, label='Predicted')
        
        optimistic_curve = self.optimistic[self.optimistic["Actual / Predicted"] == "Predicted"].hvplot(x, y_curve)
        optimistic_bar = self.optimistic[self.optimistic["Actual / Predicted"] == "Predicted"].hvplot.bar(x, y_bar, label='Optimistic')
        
        return pn.Row(actual_curve * actual_bar * predicted_curve * predicted_bar * optimistic_curve * optimistic_bar, predicted.hvplot.table()) 

i = ImpactHoursData()

pn.Row(i, i.view)

Row
    [0] Column(margin=5, name='ImpactHoursData', width=300)
        [0] StaticText(value='<b>ImpactHoursData</b>')
        [1] FloatSlider(name='Prediction', step=0.01, value_throttled=0)
    [1] ParamMethod(method)

In [31]:
import pandas as pd
import panel as pn
import os
import hvplot.pandas
APP_PATH = './'

sheets = [
    "Total Impact Hours so far", 
    "IH Predictions", 
    "#8 Jan 1",
    "#7 Dec 18",
    "#6 Dec 4",
    "#5 Nov 20",
    "#4 Nov 6",
    "#3 Oct 23",
    "#2 Oct 9",
    "#1 Sept 24",
    "#0 Sept 7 (historic)",
] + [f"#{i} IH Results" for i in range(9)]
sheets = {i:sheet for i, sheet in enumerate(sheets)}

def read_excel(sheet_name="Total Impact Hours so far", header=1, index_col=0, usecols=None) -> pd.DataFrame:
    data = pd.read_excel(
        os.path.join(APP_PATH, "data", "TEC Praise Quantification.xlsx"),
        sheet_name=sheet_name,
        engine='openpyxl',
        header=header,
        index_col=index_col,
        usecols=usecols
    ).reset_index().dropna(how='any')
    return data

## Tests
impact_hour_data_1 = read_excel()
impact_hour_data_2 = read_excel(sheet_name="IH Predictions", header=0, index_col=0, usecols='A:I').drop(index=19)
pn.Row(impact_hour_data_1.hvplot.table(), impact_hour_data_2.hvplot.table())

Row
    [0] HoloViews(Table)
    [1] HoloViews(Table)

### Impact Hours Formula
This is really cool.

In [32]:
import numpy as np
class ImpactHoursFormula(param.Parameterized):
    """
    Sem's Formula 🌱 🐝 🍯
    This formala was a collaboration of Sem and Griff for the TEC hatch impact hours formula. 
    https://forum.tecommons.org/t/impact-hour-rewards-deep-dive/90/5
    """
    total_impact_hours = param.Number(round(impact_hour_data_1['Impact Hours'].sum()), constant=True)
    max_impact_hour_rate = param.Number(100, bounds=(0,200))
    expected_raise_per_impact_hour = param.Number(25, bounds=(0,200))
#     total_xdai_raised = param.Range()

    def impact_hours_rewards(self):
        x = np.linspace(h.min_goal(), h.max_goal())

        R = self.max_impact_hour_rate

        m = self.expected_raise_per_impact_hour
        
        H = self.total_impact_hours

        y = [R* (x / (x + m*H)) for x in x]

        df = pd.DataFrame([x,y]).T
        df.columns = ['Total XDAI Raised','Impact Hour Rate']

        return df.hvplot(x='Total XDAI Raised')

In [33]:
impact_hours_formula = ImpactHoursFormula()

pn.Row(impact_hours_formula, impact_hours_formula.impact_hours_rewards)

Row
    [0] Column(margin=5, name='ImpactHoursFormula', width=300)
        [0] StaticText(value='<b>ImpactHoursFormula</b>...)
        [1] FloatInput(disabled=True, name='Total impact hours', value=2893, value_throttled=2893)
        [2] FloatSlider(end=200, name='Max impact hour rate', value=100, value_throttled=100)
        [3] FloatSlider(end=200, name='Expected raise p..., value=25, value_throttled=25)
        [4] LiteralInput(name='Total xdai raised')
    [1] ParamMethod(method)

# Target/Expected Goals
What are peoples goal target raise?

In [493]:
class CommunityTargets(param.Parameterized):
    pass

In [39]:
10000000 / 5000

2000.0